# M4d: Compositionality — Semantic Arithmetic

```
king - man + woman = queen
paris - france + germany = berlin
```

If HDC preserves these → we captured MEANING.

In [ ]:
!pip install -q sentence-transformers
print('Done')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import json
from datetime import datetime
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
np.random.seed(42)

In [ ]:
encoder = SentenceTransformer('all-mpnet-base-v2')
SEMANTIC_DIM = encoder.get_sentence_embedding_dimension()
print(f'Encoder: {SEMANTIC_DIM}d')

In [ ]:
HDC_DIM = 4096
proj = np.random.randn(SEMANTIC_DIM, HDC_DIM).astype(np.float32)
proj /= np.linalg.norm(proj, axis=0, keepdims=True)

def to_hdc(text):
    emb = encoder.encode(text)
    hdc = emb @ proj
    thr = 0.3 * np.std(hdc)
    return np.where(hdc > thr, 1, np.where(hdc < -thr, -1, 0)).astype(np.float32)

def to_float_hdc(text):
    return encoder.encode(text) @ proj

print('HDC ready')

In [ ]:
ANALOGIES = [
    ('king', 'man', 'woman', 'queen'),
    ('paris', 'france', 'germany', 'berlin'),
    ('tokyo', 'japan', 'france', 'paris'),
    ('madrid', 'spain', 'italy', 'rome'),
    ('walked', 'walk', 'swim', 'swam'),
    ('running', 'run', 'walk', 'walking'),
    ('bigger', 'big', 'small', 'smaller'),
    ('doctor', 'hospital', 'teacher', 'school'),
    ('ceo', 'company', 'captain', 'team'),
    ('puppy', 'dog', 'kitten', 'cat'),
    ('hot', 'cold', 'tall', 'short'),
    ('happy', 'sad', 'fast', 'slow'),
]
print(f'{len(ANALOGIES)} analogies')

In [ ]:
VOCAB = list(set([w for a in ANALOGIES for w in a]))
VOCAB += ['prince','princess','london','moscow','beijing','washington','ran','jumped',
          'largest','smallest','nurse','university','president','player','baby','adult',
          'warm','cool','angry','quick','boy','girl','husband','wife','brother','sister']
VOCAB = list(set(VOCAB))
print(f'Vocabulary: {len(VOCAB)} words')

In [ ]:
print('Encoding vocabulary...')
vocab_emb = {w: encoder.encode(w) for w in VOCAB}
vocab_hdc = {w: to_hdc(w) for w in VOCAB}
vocab_hdc_float = {w: to_float_hdc(w) for w in VOCAB}
print(f'Done: {len(VOCAB)} words')

In [ ]:
def find_nearest(query_vec, vocab_dict, exclude=None, top_k=5):
    exclude = exclude or []
    sims = [(w, cosine_similarity([query_vec], [v])[0][0]) for w, v in vocab_dict.items() if w not in exclude]
    sims.sort(key=lambda x: x[1], reverse=True)
    return sims[:top_k]

print('Nearest to king:', find_nearest(vocab_hdc_float['king'], vocab_hdc_float, ['king'])[:3])

In [ ]:
def test_analogy(a, b, c, expected, vocab_dict, requantize=False):
    result_vec = vocab_dict[a] - vocab_dict[b] + vocab_dict[c]
    if requantize:
        thr = 0.3 * np.std(result_vec)
        result_vec = np.where(result_vec > thr, 1, np.where(result_vec < -thr, -1, 0))
    nearest = find_nearest(result_vec, vocab_dict, exclude=[a,b,c], top_k=5)
    top_words = [w for w,s in nearest]
    return {'analogy': f'{a} - {b} + {c}', 'expected': expected, 'got': nearest[0][0],
            'top5': top_words, 'hit1': nearest[0][0] == expected, 'hit5': expected in top_words}

In [ ]:
print('='*60)
print('BASELINE: Original Embeddings (768d)')
print('='*60)
base_results = [test_analogy(a,b,c,d,vocab_emb) for a,b,c,d in ANALOGIES]
for r in base_results:
    s = '✓' if r['hit1'] else ('~' if r['hit5'] else '✗')
    print(f"{s} {r['analogy']} = {r['got']} (expected: {r['expected']})")
base_top1 = np.mean([r['hit1'] for r in base_results])
base_top5 = np.mean([r['hit5'] for r in base_results])
print(f'\nBaseline Top-1: {base_top1:.1%}, Top-5: {base_top5:.1%}')

In [ ]:
print('\n' + '='*60)
print('FLOAT HDC (4096d)')
print('='*60)
float_results = [test_analogy(a,b,c,d,vocab_hdc_float) for a,b,c,d in ANALOGIES]
for r in float_results:
    s = '✓' if r['hit1'] else ('~' if r['hit5'] else '✗')
    print(f"{s} {r['analogy']} = {r['got']} (expected: {r['expected']})")
float_top1 = np.mean([r['hit1'] for r in float_results])
float_top5 = np.mean([r['hit5'] for r in float_results])
print(f'\nFloat HDC Top-1: {float_top1:.1%}, Top-5: {float_top5:.1%}')

In [ ]:
print('\n' + '='*60)
print('TERNARY HDC (4096d)')
print('='*60)
tern_results = [test_analogy(a,b,c,d,vocab_hdc,requantize=True) for a,b,c,d in ANALOGIES]
for r in tern_results:
    s = '✓' if r['hit1'] else ('~' if r['hit5'] else '✗')
    print(f"{s} {r['analogy']} = {r['got']} (expected: {r['expected']})")
tern_top1 = np.mean([r['hit1'] for r in tern_results])
tern_top5 = np.mean([r['hit5'] for r in tern_results])
print(f'\nTernary HDC Top-1: {tern_top1:.1%}, Top-5: {tern_top5:.1%}')

In [ ]:
print('\n' + '='*60)
print('SUMMARY')
print('='*60)
print(f'{"Method":<30} {"Top-1":>10} {"Top-5":>10} {"vs Base":>10}')
print('-'*60)
print(f'{"Original (768d float)":<30} {base_top1:>10.1%} {base_top5:>10.1%} {"baseline":>10}')
print(f'{"Float HDC (4096d)":<30} {float_top1:>10.1%} {float_top5:>10.1%} {float_top5/base_top5 if base_top5>0 else 0:>10.1%}')
print(f'{"Ternary HDC (4096d)":<30} {tern_top1:>10.1%} {tern_top5:>10.1%} {tern_top5/base_top5 if base_top5>0 else 0:>10.1%}')
print('\n✓ = exact match, ~ = in top-5, ✗ = miss')

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

ax = axes[0]
methods = ['Original\n(768d)', 'Float HDC\n(4096d)', 'Ternary\n(4096d)']
x = np.arange(3)
w = 0.35
bars1 = ax.bar(x - w/2, [base_top1, float_top1, tern_top1], w, label='Top-1', color='steelblue')
bars2 = ax.bar(x + w/2, [base_top5, float_top5, tern_top5], w, label='Top-5', color='lightblue')
ax.set_ylabel('Accuracy')
ax.set_title('Semantic Arithmetic Preservation')
ax.set_xticks(x)
ax.set_xticklabels(methods)
ax.legend()
ax.set_ylim(0, 1.1)
for b in list(bars1)+list(bars2): ax.text(b.get_x()+b.get_width()/2, b.get_height()+0.02, f'{b.get_height():.0%}', ha='center', fontsize=9)

ax = axes[1]
hits = np.array([[r['hit1'] for r in base_results], [r['hit1'] for r in float_results], [r['hit1'] for r in tern_results]])
im = ax.imshow(hits, cmap='RdYlGn', aspect='auto', vmin=0, vmax=1)
ax.set_yticks([0,1,2])
ax.set_yticklabels(['Original', 'Float HDC', 'Ternary'])
ax.set_xticks(range(len(ANALOGIES)))
ax.set_xticklabels([a[0][:4] for a in ANALOGIES], rotation=45, ha='right')
ax.set_title('Per-Analogy (green=correct)')

plt.tight_layout()
plt.savefig('m4d_compositionality.png', dpi=150)
plt.show()

In [ ]:
output = {'experiment': 'M4d Compositionality', 'paper': '...Until We Found Meaning',
          'num_analogies': len(ANALOGIES), 'vocab_size': len(VOCAB),
          'results': {'baseline': {'top1': float(base_top1), 'top5': float(base_top5)},
                      'float_hdc': {'top1': float(float_top1), 'top5': float(float_top5)},
                      'ternary_hdc': {'top1': float(tern_top1), 'top5': float(tern_top5)}},
          'retention_rate': float(tern_top5/base_top5) if base_top5 > 0 else 0,
          'timestamp': datetime.now().isoformat()}
with open('m4d_compositionality.json', 'w') as f: json.dump(output, f, indent=2)
print(json.dumps(output, indent=2))

In [ ]:
print('\n' + '='*60)
if tern_top5/base_top5 > 0.8 if base_top5 > 0 else tern_top5 > 0.5:
    print('✅ SUCCESS! HDC preserves semantic arithmetic')
    print('   "king - man + woman = queen" works in ternary!')
else:
    print('⚠️ Partial preservation')
print('='*60)

In [ ]:
from google.colab import files
files.download('m4d_compositionality.json')
files.download('m4d_compositionality.png')